In [1]:
import numpy as np 
import pandas as pd 
from scipy import stats
import calendar
import matplotlib as plt
import category_encoders as ce
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import svm

In [2]:
pub_df = pd.read_csv('data/ks-projects-201801.csv')
ins_df = pd.read_csv('data/insurance_event.csv')

In [28]:
"""simplified extension of the replace function in python"""

def replacen(text, kwargs):
    """any single character of `text` in `kwarg.keys()` is replaced by `kwarg[key]`
    >>> consonants = replacen('abcdefghijklmnopqrstuvwxyz', {'aeiou':''})
    """
    
    try:
        text = [str(i) for i in text]
    except (ValueError, TypeError):
        raise TypeError("`text` parameter must have valid str type")
        
    #check the contents of each key, make sure there's no overlap:
    collisions = any_key_collisions(kwargs)
    if collisions:
        raise KeyError("keys have duplicate find-replace strings: '%s'" % collisions)
    
    #bring all keys together during character comparisons
    all_keys = [[ix for ix in i] for i in kwargs.keys()]
    for idx, i in enumerate(text):
        for key in all_keys:
            if i in ''.join(key):
                text[idx] = kwargs.get(''.join(key))
    return ''.join(text)

def any_key_collisions(dictionary):
    """ensures no keys contain any other key element, across all keys"""
    members = [i for i in dictionary.keys()]
    dups = []
    for idx, _ in enumerate(members):
        candidate = members[idx * -1]
        if candidate in members[: idx * -1]:
            dups.append(candidate)
    return ''.join(set(dups))

def get_string(str):
    return replacen(str, {'0':'a','1':'b','2':'c','3':'d','4':'e','5':'f','6':'g','7':'h','8':'i','9':'j'})

In [39]:
ins_df['Event_Code'] = ins_df.Event_Code.apply(get_string)
ins_df['Event_Code'].head(100)

0     ihah
1     hcgb
2     bjgh
3     hbhc
4     ibaa
5     ibaa
6     jjdj
7     jjcb
8     dbgh
9     bjha
10    eadj
11    cgaf
12    hifb
13    cbbb
14    jjci
15    hdag
16    hiid
17    ehhj
18    dchd
19    dhac
20    jajj
21    hebi
22    eabb
23    chfg
24    icfh
25    cfaa
26    jjcb
27    bjgc
28    dcga
29    echd
      ... 
70    efji
71    ihah
72    Ahad
73    cfae
74    Vfeb
75    jjci
76    hbac
77    jjcb
78    dfba
79    caha
80    ihai
81    hcec
82    dced
83    jjcb
84    efij
85    ddjh
86    ifgb
87    iged
88    dccb
89    Aegf
90    hcef
91    iebg
92    hiha
93    dced
94    jjcb
95    cfic
96    chhc
97    fjfa
98    gbae
99    ddcf
Name: Event_Code, Length: 100, dtype: object

In [32]:
new_col = []
for row in pub_df['category']:
    new_col.append(row)
for row in ins_df['Event_Code'].head(1000):
    new_col.append(row)    
new_col=set(new_col)
new_col=list(new_col)

In [38]:
col_df = pd.DataFrame({'Test':new_col})
col_df.head()
#col_df.shape

,Test
0,ibaa
1,Drinks
2,Food
3,Drama
4,jdcj


In [5]:
def get_no_of_collisions(df,column,n):
    fh = FeatureHasher(n_features=n, input_type='string')
    hashed_features = fh.fit_transform(df[column].drop_duplicates()).toarray()
    fh_df = pd.DataFrame(hashed_features)
    g = fh_df.groupby(fh_df.columns.tolist())
    return len(fh_df) - g.ngroups

In [36]:
n_limit = {2,4,6,8,10,16,32,64,128,256,512,1024,2048,4096}
for n in n_limit:
    print("Collision for column {} when n_features = {} is : {} ".format('Test',n,get_no_of_collisions(col_df,'Test',n)))

Collision for column Test when n_features = 32 is : 198 
Collision for column Test when n_features = 64 is : 195 
Collision for column Test when n_features = 2 is : 717 
Collision for column Test when n_features = 128 is : 195 
Collision for column Test when n_features = 4 is : 615 
Collision for column Test when n_features = 256 is : 193 
Collision for column Test when n_features = 6 is : 503 
Collision for column Test when n_features = 512 is : 193 
Collision for column Test when n_features = 8 is : 465 
Collision for column Test when n_features = 1024 is : 193 
Collision for column Test when n_features = 10 is : 443 
Collision for column Test when n_features = 2048 is : 193 
Collision for column Test when n_features = 4096 is : 193 
Collision for column Test when n_features = 16 is : 287 
